# Importando bibliotecas que serão utilizadas no código

In [47]:
# IMPORTAR BIBLIOTECAS NECESSÁRIAS

!pip install oracledb
import oracledb

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from IPython.display import display

----
----

### Criando conexão com o banco de dados oracle, utilizando a biblioteca

```
!pip install oracledb
import oracledb
```



In [48]:
# CONFIGURAÇÃO DE CONEXÃO COM O BANCO DE DADOS ORACLE
connection = oracledb.connect(user='rm551812', password='fiap23', dsn='oracle.fiap.com.br:1521/orcl')

----
----

### Puxar os dados das tabelas, no banco de dados oracle e depois exibir os dados no console

> IA_PRODUTO

> IA_CLIENTE

> IA_HISTORICO_PESQUISA





In [49]:
# Função para carregar os dados de produtos e características dos usuários
def load_data():
    query_produtos = "SELECT * FROM IA_PRODUTO"
    query_clientes = "SELECT * FROM IA_CLIENTE"
    query_historico = "SELECT * FROM IA_HISTORICO_PESQUISA"

    df_produtos = pd.read_sql(query_produtos, con=connection)
    df_clientes = pd.read_sql(query_clientes, con=connection)
    df_historico = pd.read_sql(query_historico, con=connection)

    return df_produtos, df_clientes, df_historico

# Acionando a função
df_produtos, df_clientes, df_historico = load_data()

# Exibindo os dados
display(df_produtos)
display(df_clientes)
display(df_historico)

<ipython-input-49-bb14d6cb40ff>:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_produtos = pd.read_sql(query_produtos, con=connection)
<ipython-input-49-bb14d6cb40ff>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_clientes = pd.read_sql(query_clientes, con=connection)
<ipython-input-49-bb14d6cb40ff>:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_historico = pd.read_sql(query_historico, con=connection)


,ID,NOME_PRODUTO,DESCRICAO,VALOR,IMG,DT_FABRICACAO,DT_VALIDADE,DT_CADASTRO
0,1,Shampoo Revitalizante,Shampoo para cabelos secos e danificados,15.99,img/shampoo1.jpg,2024-05-18,2025-05-18,2024-05-18 13:12:29
1,2,Creme Facial Antioxidante,Creme hidratante para peles sensíveis,29.99,img/creme1.jpg,2024-03-10,2024-12-10,2024-05-18 13:12:29
2,3,Desodorante Roll-On,Desodorante antitranspirante com fragrância suave,8.50,img/desodorante1.jpg,2023-11-20,2025-11-20,2024-05-18 13:12:29
3,4,Loção Corporal Hidratante,Loção corporal com aloe vera para peles secas,12.75,img/locao1.jpg,2024-01-05,2024-12-05,2024-05-18 13:12:29
4,5,Creme Dental Clareador,Creme dental com carvão ativado para dentes ma...,6.99,img/cremedental1.jpg,2024-02-15,2025-02-15,2024-05-18 13:12:29
5,6,Perfume Floral,Perfume feminino com notas de flores brancas,49.99,img/perfume1.jpg,2023-10-01,2025-10-01,2024-05-18 13:12:30
6,7,Gel de Limpeza Facial,Gel de limpeza facial para peles mistas,10.50,img/gel1.jpg,2024-04-03,2025-04-03,2024-05-18 13:12:30
7,8,Protetor Solar FPS 50,Protetor solar resistente à água e com alta pr...,22.25,img/protetorsolar1.jpg,2024-05-01,2025-05-01,2024-05-18 13:12:30
8,9,Sabonete Líquido Neutro,Sabonete líquido neutro para uso diário,5.99,img/sabonete1.jpg,2024-03-20,2025-03-20,2024-05-18 13:12:30
9,10,Creme de Barbear Suavizante,Creme de barbear para peles sensíveis com efei...,7.50,img/cremebarbear1.jpg,2024-06-10,2025-06-10,2024-05-18 13:12:30


,ID,NOME_CLIENTE,EMAIL,CPF,COR_PELE,MARCA_PREFERIDA,ESTADO_CIVIL,SENHA,DT_CADASTRO,DT_NASCIMENTO,ID_GENERO
0,1,Ana Oliveira,ana@example.com,12345678901,branco,Natura,Solteiro(a),senha123,2023-05-18,1987-06-20,2
1,2,Pedro Silva,pedro@example.com,98765432109,moreno,Avon,Casado(a),senha456,2022-07-20,1985-09-15,1
2,3,Carla Santos,carla@example.com,65432198704,negro,O Boticário,Solteiro(a),senha789,2024-04-10,2000-12-25,2
3,4,Mariana Oliveira,mariana@example.com,45678912305,pardo,MAC Cosmetics,Divorciado(a),senha101112,2023-03-05,1978-07-18,2
4,5,Lucas Souza,lucas@example.com,98765432102,moreno,Eudora,Solteiro(a),senha131415,2024-09-22,1995-11-10,1


,ID,CLIENTE_ID,PRODUTO_ID
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,4
4,5,5,5
5,6,1,6


----
----
### Processar os dados obtidos nos SELECTS

In [50]:
# PRÉ-PROCESSAMENTO DOS DADOS
def preprocess_data(df_produtos, df_clientes, df_historico):
    # Merge dos dados relevantes
    df = pd.merge(df_historico, df_clientes, left_on='CLIENTE_ID', right_on='ID')
    df = pd.merge(df, df_produtos, left_on='PRODUTO_ID', right_on='ID')

    # Selecionar apenas as colunas relevantes para a recomendação
    features = ['COR_PELE', 'ESTADO_CIVIL', 'MARCA_PREFERIDA', 'NOME_PRODUTO']  # Nome do produto adicionado
    df = df[features]

    # Convertendo colunas categóricas em variáveis dummy
    df = pd.get_dummies(df, columns=['COR_PELE', 'ESTADO_CIVIL', 'MARCA_PREFERIDA'])

    return df

# ACIONANDO A FUNÇÃO DE PROCESSAMENTO DOS DADOS
df = preprocess_data(df_produtos, df_clientes, df_historico)


----
----

### Fazendo divisao entre dados de TREINO e dados de TESTE, para realizar o modelo de treinamento.

In [51]:
# DIVIDINDO OS DADOS EM TREINO E TESTE
X = df.drop('NOME_PRODUTO', axis=1)
y = df['NOME_PRODUTO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TREINANDO UM MODELO DE CLASSIFICAÇÃO
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

----
----

### Buscando dados do usuário no banco e enviando para a função de recomendação de produtos

### Função para recomendar o produto, de acordo com a analise e o treinamento do modelo

In [54]:
# FUNÇÃO PARA RECOMENDAR PRODUTOS
def recommend_product(client_data):
    client_df = pd.DataFrame([client_data])
    client_df = pd.get_dummies(client_df)

    # Alinhar as colunas do client_df com as colunas usadas no treinamento
    client_df = client_df.reindex(columns=X.columns, fill_value=0)

    recommended_product = model.predict(client_df)
    return recommended_product[0]


# Buscar dados de um cliente específico no banco de dados
def get_client_data(client_id):
    query_cliente = f"SELECT * FROM IA_CLIENTE WHERE ID = {client_id}"
    df_cliente = pd.read_sql(query_cliente, con=connection)

    if df_cliente.empty:
        raise ValueError("Cliente não encontrado.")

    # Mapeando os dados do cliente para o formato esperado pela função recommend_product
    client_data = {
        'cor_pele': df_cliente.iloc[0]['COR_PELE'],
        'estado_civil': df_cliente.iloc[0]['ESTADO_CIVIL'],
        'marca_preferida': df_cliente.iloc[0]['MARCA_PREFERIDA']
    }

    return client_data

# Exemplo de uso
try:
    client_id = 1  # Substitua pelo ID do cliente que você deseja buscar
    client_data = get_client_data(client_id)
    recommended_product = recommend_product(client_data)
    print(f"\n\nProduto recomendado: {recommended_product}\n\n")
except ValueError as e:
    print(e)



Produto recomendado: Loção Corporal Hidratante




<ipython-input-54-a80a3ee8deeb>:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cliente = pd.read_sql(query_cliente, con=connection)
